In [12]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# ——————————————————————————————————
# 1) 데이터 로드
# ——————————————————————————————————
# TS (TimeSformer) 임베딩
X_ts_train = np.load('embbeding_data/timesformer/train/embeddings.npy')
y_train     = np.load('embbeding_data/timesformer/train/labels.npy').ravel()
X_ts_test  = np.load('embbeding_data/timesformer/test/embeddings.npy')
y_test     = np.load('embbeding_data/timesformer/test/labels.npy').ravel()

# ST (ST-GCN) 임베딩 (3D → 2D flatten)
X_st_train = np.load('embbeding_data/stgcnpp/train/embeddings.npy')
X_st_test  = np.load('embbeding_data/stgcnpp/test/embeddings.npy')
# 세그먼트 축(1)을 평균 처리
X_st_train = X_st_train.mean(axis=1)
X_st_test  = X_st_test.mean(axis=1)

# ——————————————————————————————————
# 2) TS-only 성능
# ——————————————————————————————————
clf_ts = LogisticRegression(max_iter=1000).fit(X_ts_train, y_train)
pred_ts_train = clf_ts.predict(X_ts_train)
prob_ts_train = clf_ts.predict_proba(X_ts_train)[:,1]
pred_ts_test  = clf_ts.predict(X_ts_test)
prob_ts_test  = clf_ts.predict_proba(X_ts_test)[:,1]

print("TS-only Train  acc:", accuracy_score(y_train, pred_ts_train))
print("TS-only Train  AUC:", roc_auc_score(y_train, prob_ts_train))
print("TS-only Test   acc:", accuracy_score(y_test,  pred_ts_test))
print("TS-only Test   AUC:", roc_auc_score(y_test,  prob_ts_test))

# ——————————————————————————————————
# 3) ST-only 성능
# ——————————————————————————————————
clf_st = LogisticRegression(max_iter=1000).fit(X_st_train, y_train)
pred_st_train = clf_st.predict(X_st_train)
prob_st_train = clf_st.predict_proba(X_st_train)[:,1]
pred_st_test  = clf_st.predict(X_st_test)
prob_st_test  = clf_st.predict_proba(X_st_test)[:,1]

print("\nST-only Train acc:", accuracy_score(y_train, pred_st_train))
print("ST-only Train AUC:", roc_auc_score(y_train, prob_st_train))
print("ST-only Test  acc:", accuracy_score(y_test,  pred_st_test))
print("ST-only Test  AUC:", roc_auc_score(y_test,  prob_st_test))

# ——————————————————————————————————
# 4) Fusion 성능 (두 임베딩 concatenate)
# ——————————————————————————————————
X_f_train = np.hstack([X_ts_train, X_st_train])
X_f_test  = np.hstack([X_ts_test,  X_st_test])
clf_f = LogisticRegression(max_iter=1000).fit(X_f_train, y_train)
pred_f_train = clf_f.predict(X_f_train)
prob_f_train = clf_f.predict_proba(X_f_train)[:,1]
pred_f_test  = clf_f.predict(X_f_test)
prob_f_test  = clf_f.predict_proba(X_f_test)[:,1]

print("\nFusion Train acc:", accuracy_score(y_train, pred_f_train))
print("Fusion Train AUC:", roc_auc_score(y_train, prob_f_train))
print("Fusion Test  acc:", accuracy_score(y_test,  pred_f_test))
print("Fusion Test  AUC:", roc_auc_score(y_test,  prob_f_test))


TS-only Train  acc: 1.0
TS-only Train  AUC: 1.0
TS-only Test   acc: 0.40569395017793597
TS-only Test   AUC: 0.28654703974047036

ST-only Train acc: 0.6538461538461539
ST-only Train AUC: 0.689532208588957
ST-only Test  acc: 0.5124555160142349
ST-only Test  AUC: 0.4280210867802109

Fusion Train acc: 1.0
Fusion Train AUC: 1.0
Fusion Test  acc: 0.3879003558718861
Fusion Test  AUC: 0.27808191403081917


In [9]:
import numpy as np
from collections import Counter

# --- 레이블 데이터 로드 및 분포 파악 함수 정의 ---
def analyze_label_distribution(file_path, data_name):
    """
    지정된 경로의 npy 파일에서 레이블을 로드하고 분포를 분석하여 출력합니다.
    """
    print(f"--- {data_name} 레이블 분포 ---")
    try:
        labels = np.load(file_path).ravel()
        label_distribution = Counter(labels)

        # 레이블 값을 기준으로 오름차순 정렬하여 출력
        for label, count in sorted(label_distribution.items()):
            print(f"  레이블 '{label}': {count}개")
        print(f"  총 레이블 개수: {len(labels)}개")
    except FileNotFoundError:
        print(f"  오류: '{file_path}' 파일을 찾을 수 없습니다. 경로를 확인해 주세요.")
    except Exception as e:
        print(f"  오류: '{file_path}' 파일 처리 중 예상치 못한 문제가 발생했습니다: {e}")
    print("\n" + "="*50 + "\n")


# --- 각 레이블 데이터에 대해 분석 실행 ---

# 1. TimesFormer 훈련 데이터 레이블 (y_ts_train)
analyze_label_distribution(
    r'D:\Jabez\golf\fusion\embbeding_data\timesformer\train\labels.npy',
    'TimesFormer 훈련 데이터 (y_ts_train)'
)

# 2. TimesFormer 테스트 데이터 레이블 (y_ts_test)
analyze_label_distribution(
    r'D:\Jabez\golf\fusion\embbeding_data\timesformer\test\labels.npy',
    'TimesFormer 테스트 데이터 (y_ts_test)'
)

# 3. ST-GCN 훈련 데이터 레이블 (y_st_train)
analyze_label_distribution(
    r'D:\Jabez\golf\fusion\embbeding_data\stgcnpp\train\labels.npy',
    'ST-GCN 훈련 데이터 (y_st_train)'
)

# 4. ST-GCN 테스트 데이터 레이블 (y_st_test)
analyze_label_distribution(
    r'D:\Jabez\golf\fusion\embbeding_data\stgcnpp\test\labels.npy',
    'ST-GCN 테스트 데이터 (y_st_test)'
)

--- TimesFormer 훈련 데이터 (y_ts_train) 레이블 분포 ---
  레이블 '0': 326개
  레이블 '1': 480개
  총 레이블 개수: 806개


--- TimesFormer 테스트 데이터 (y_ts_test) 레이블 분포 ---
  레이블 '0': 137개
  레이블 '1': 138개
  총 레이블 개수: 275개


--- ST-GCN 훈련 데이터 (y_st_train) 레이블 분포 ---
  레이블 '0': 326개
  레이블 '1': 480개
  총 레이블 개수: 806개


--- ST-GCN 테스트 데이터 (y_st_test) 레이블 분포 ---
  레이블 '0': 137개
  레이블 '1': 138개
  총 레이블 개수: 275개




In [8]:
import pickle
from pathlib import Path
from collections import Counter

# 1) test pkl 경로 설정
PKL_PATH = Path(r"D:\golfDataset\dataset\test\crop_pkl\skeleton_dataset_test.pkl")

# 2) PKL 로드
with open(PKL_PATH, 'rb') as f:
    data = pickle.load(f)

annotations = data['annotations']

# 3) 샘플 10개만 출력
print("📦 Test 샘플 10개 미리보기:")
for sample in annotations[:10]:
    print(f" - ID: {sample['frame_dir']}, Label: {sample['label']}")

# 4) 전체 라벨 통계
labels = [sample['label'] for sample in annotations]
counter = Counter(labels)
print("\n📊 전체 레이블 분포:")
print(f"  0 (bad/worst): {counter[0]}")
print(f"  1 (normal/good/great): {counter[1]}")


📦 Test 샘플 10개 미리보기:
 - ID: 20201124_General_037_DOS_A_M40_MS_001_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_003_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_006_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_010_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_011_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_016_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_018_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_022_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_030_crop, Label: 1
 - ID: 20201124_General_037_DOS_A_M40_MS_031_crop, Label: 1

📊 전체 레이블 분포:
  0 (bad/worst): 137
  1 (normal/good/great): 138
